In [38]:
import pandas as pd
import numpy as np
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os

In [39]:
# Inputs and Parameters

# File Path
filePath = "C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/PER3_16x10E.dat"
file = "PER3_20x10E"
fileName = 'C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/' + file + '.dat'

# Constants
R = 8.3132                       # Gas Constant (J/K-mol)
M = 0.02896                      # Molar Mass of Earth's Air (kg)
g = 9.806                        # Force of Gravity (N)
P0 = 101325                      # Pressure at Sea Level (Pa) - Double Check Units with this value

# Starting Point
propDia = 20.0 * 0.0254          # Prop Diameter (in)
alt = 1289 * 0.3048              # Altitude w/ coversion (m)
t = 15+273.15-6.5*alt*.001       # Ambient Temp (K)
Pressure = P0 * np.exp(-M * g * h / (R * T))    # Pressure at Alt (Pa)
density = Pressure * M / (R * T) # Air Density ()

throttle = 100.0                 # Throttle value (0 to 100)

# ESC
eCurrMax = 120.0                 # Max current of esc (A      -- Double Check these
eRes = 0.004                     # esc Resistance (milliOhms)      values!!

# Battery Cell -- From LiPos
cells = 6.0                      # Num cells
batVolt = 3.7                    # LiPo per-cell Voltage (V)
batRes = 0.026                   # bat Resistance (milliOhms)

# Motor
kv = 295.0                       # RPM per Volt
nLCurr = 1.7                     # No Load Current (A)
nLVolt = 8.4                     # No Load Voltage (V)
mRes = 0.015                     # Motor Resistnace (milliOhms)
mWatt = 2000.0                   # Max Watts of Motor (A)

In [40]:
""" Pulling Data """

# Set Column Index for Importing Data
header_list = [str(x) for x in range(13)]

# Read file
propData = pd.read_csv(fileName, sep = '\s+', names = header_list)
propData = propData.to_numpy()
df = pd.DataFrame(propData)
df.to_csv(index = False)

# Find All Rows w/ RPM Data
searchString = "RPM"
mask = np.column_stack([df[col].str.contains(r'' + searchString, na = False)
                        for col in df])
RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis = 1)], axis = 1)

In [41]:
"""
Create New Table to bin RPM Values into Groups
Remove Characters from PROP RPM
Change to Numeric Values for Analysis
"""

# First Find Column w/ Numbers
RPMs = RPMdata.apply(pd.to_numeric, errors = 'ignore')

# Second Remove All non-int64 Columns
RPMs = RPMs.select_dtypes(include = ['int64'])

# Third Get First RPM Value Index
a = RPMs.columns.values
indexinRPM = RPM[a[0]].index[0]

# Delete Unnecessary Top Rows
RPMdf = df.iloc[indexinRPM:]

# Get Max RPM and Index
maxRPM = RPMs.max()
indexMaxRPM = RPMs[a[0]].idxmax()

# Get Index of Each RPM Value to Bin Data
RPMindex = RPMs.index

# Clear Columns w/ Mostly NA
RPMdf = pd.DataFrame.dropna(RPMdf, axis = 'columns', thresh = 10)

# Add Index of Last Row of Database ot RPM DF to Bin
lastIndex = RPMdf.iloc[[-1]].index.values[0]

# Get Index of RPMs ot Group by RPM
RPMindex = np.append(RPMindex, lastIndex)
RPMlabel = RPMs[a[0]].to_list()

# Label Columns - Assumes consistantcy in value-to-columns, V in first column
Units = RPMdf[RPMdf[0].str.match('V')]

# First Row of "V"
unitsRow = Units[0].index[0]

# Takes First Row, Sets as Column Headers
RPMdf.columns = RPMdf.loc[unitsRow]

# Removes All Non-Numeric Rows
RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()

# Group Each RPM
RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex,
                             right = False, labels = RPMlabel))

I'm done!


In [ ]:
""" Notes & Shortened Terms:
        Curr = Current
        Res = Resistance
        Batt = Battery
        Volt = Voltage
        v = Voltage (when put before term)
        p = Power (when put before term)
        Amps = Amperage
        df = Data Frame
        J = Velocity / (PRs * Diameter (m))
"""

In [ ]:
""" First Calculations """

RPMgroups = np.array(list(RPMdf.groups))

# allAmps array w/ Expected Amp Draw at Max Static Thrust at each velocity
allAmps = []

for groups in RPMgroups:
    RPMgroup = RPMdf.get_group(groups)
    PWR = RPMgroup['PWR'].iloc[0] * 745.7  # Converting HP to Watts
    allAmps.append(PWR / (cells * BatVolt))
    
def RPMfinder(motorAmpGuess, batAmpGuess, Output, Velocity):
    
    # Main Calc for Iteration of pMotorOut
    def pMotorOutCalc(motorAmp, batAmp):
    
        vBat = cells * batVolt - cells * batRes * batAmp         # Total Bat Volts w/ Volt Loss
        vTerm = (vBat - eRes * batAmp) * Throttle / 100          # Volt Drop from esc/Throttle Val
        vDropMotor = motorAmp * mRes                             # Volt Drop from Motor
        pMotorIn = vTerm * batAmp                                # Power Motor Uses during Opt
        vEMF = vTerm - vDropMotor                                # Voltage Motor after losses
        pMotorOut = vEMF * (motorAmp - nLCurr)                   # Power Motor Generates (Voltage of Motor * (Amps of Motor - No Load Motor Amps))
        pMotorIn = vTerm * (motorAmp - nLCurr)                   # Power Motor Receives

        return pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF
    
    pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF = pMotorOutCalc(motorAmpGuess, batAmpGues)
    
    RPM = vEMF * kv                # Revolutions/Min of Motor
    RPS = RPM / 60                 # Revolutions/s of Motor
    
    
    if (batAmp > batVolt/batRes):
        raise Exception("Batteries can't supply this much current: " + str(batAmp))
    if (battAmpGues > eCurrMax):
        raise Exception("ESC will be overloaded, chose larger ESC: " + str(Amps))
    if (RPM < 0):
        raise Exception("RPM is negative, something is wrong: " + str(RPM))
    
    
    # Find Closest Data to RPM Values, above and below
    RPMgroups = np.array(list(RPMdf.groups))
    
    if (RPM > RPMgroups.max()):
        raise Exception("RPM max limit has been reached")
    if (RPM < RPMgroups.min*()):
        raise Exception('RPM minimum limit reached')
    
    bottomRPM = RPMgroups[RPMgroups < RPM].max()
    topRPM = RPMgroups[RPMgroups > RPM].min()
    
    # Get Bottom/Top RPM Groups
    bottomRPMddata = RPMdf.get_group(bottomRPM).reset_index()
    topRPMdata = RPMdf.get_group(top.RPM).reset_index()
    
    velocityMPH = velocity * 2.23694      # Conversion to MPH
    
    # Finding Relevant Indices
    bottomRPMClosestVel = abs(bottomRPMdata['V'] - velocityMPH).idxmin()
    topRPMClosestVel = abs(topRPMdata['V'] - velocityMPH).idxmax()
    
    # Finding CT
    bottomCt = bottomRPMdata['Ct'].iloc[bottomRPMClosestVel]
    topCt = topRPMdata['Ct'].iloc[topRPMClosestVel]
    
    # Finding Cp
    bottomCp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVel]
    topCp = topRPMdata['Cp'].iloc[topRPMClosestVel]
    
    # Interpolate to get Cp/Ct for Motor RPM
    Cp = bottomCp + (RPM - bottomRPM) * (topCp - bottomCp) / (topRPM - bottomRPM)
    Ct = bottomCt + (RPM - bottomRPM) * (topCt - bottomCt) / (topRPM - bottomRPM)
    torque = bottomTorque + (RPM - bottomRPM) * (topTorque - bottomTorque) / (topRPM - bottomRPM)
    
    # Finding Torque
    bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVel]
    topTorque = topRPMdata['Torque'].iloc[topRPMClosestVel]
    
    # Prop Power/Thrust Calc
    pProp = Density * RPS**3 * propDiameter**5 * Cp         # Power Prop requires in kg * m^2 / s^3 (W)
    ThrustProp = Density * RPS**2 * propDiameter**4 * Ct    # Thrust Prop generates in kg*m/2  (N)
    
    if Output = 1:
        print('Export final parameters here')
        
    return pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMP, RPM, torque

In [ ]:
""" Final Calculations """

InitMotorGuess = nLCurr + 0.01    # Make First Motor Amp Draw Guess
InitBatGuess = nLCurr + 0.01      # No Load Amp Draw for Motor

# Dataframe for Storing Final Results
resultProp = pd.DataFrame(columns = [ "Battery Current (A)", "RPM (rev/min)", "Thrust (N)", "Power (W)", "Efficiency", "Torque (in-lbf)"])
resultProp.columns.name = 'Velocity (m/s)'

powerTolerance = 0.01 # Tolerance of power to Motor to Prop

velocityPosition = 0 # VelocityArray Position
velocity = velocityArray[velocityPosition] # Current Velocity in m/s

pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, Torque = RPMfinder(InitM)